In [10]:
import pandas as pd
import numpy as np

# 假设 time_series 是已经加载的 DataFrame
# 这里用示例数据来模拟真实数据
# data = {
#     'Timestamp': ['2021-07-02 16:35:25', '2021-07-02 16:36:25', '2021-07-02 16:37:25', '2021-07-02 16:38:25', '2021-07-02 16:39:25'],
#     'Sleepstate': [4, 4, 4, 4, 4],
#     'Breath': [16, 16, 16, 16, 16],
#     'Heartrate': [78, 76, 74, 74, 73],
#     'Movement': [2, 0, 0, 0, 1],
#     'Source': [1, 1, 1, 1, 1]
# }
kailuan_data = pd.read_csv("./combined_sheets.csv")
# time_series = pd.DataFrame(kailuan_data)

# 将数据按照 Source 分组
grouped = kailuan_data.groupby('Source')

# 提取特征列
features = ['Sleepstate', 'Breath', 'Heartrate', 'Movement']

In [13]:
# 重新计算每个样例的时间步数（考虑到了数据过滤）
filtered_time_steps_per_source = kailuan_data.groupby('Source').size()

# 重新筛选出有效的 Source（确保每个分组至少有一个数据点）
valid_sources = filtered_time_steps_per_source[filtered_time_steps_per_source > 500].index

# 重新获取有效的数据
valid_filtered_time_series = kailuan_data[kailuan_data['Source'].isin(valid_sources)]

# 分组
valid_filtered_grouped = valid_filtered_time_series.groupby('Source')

# 计算平均时间步数（此时应该没有 NaN 值）
average_time_steps = int(valid_filtered_grouped.size().mean())

# 初始化新的三维数组（考虑到了有效的样例数）
valid_sources_count = len(valid_sources)
three_dim_array_valid = np.zeros((valid_sources_count, average_time_steps, len(features)))

# 填充新的三维数组
for i, (source, group) in enumerate(valid_filtered_grouped):
    data = group[features].values
    current_steps = data.shape[0]

    if current_steps < average_time_steps:
        fill_values = {
            'Breath': group['Breath'].mean(),
            'Heartrate': group['Heartrate'].mean(),
            'Sleepstate': group['Sleepstate'].median(),
            'Movement': 0
        }
        fill_array = np.array([[fill_values[feature] for feature in features]] * (average_time_steps - current_steps))
        full_data = np.vstack((data, fill_array))
    else:
        full_data = data[:average_time_steps, :]

    three_dim_array_valid[i, :, :] = full_data

three_dim_array_valid.shape  # 显示新的三维数组的形状

(129, 1075, 4)